In [2]:
import numpy as np
import pandas as pd
import dateutil as dtutil
import pytz
import six


In [4]:
np.__version__

'1.19.5'

In [5]:
pd.__version__

'1.1.5'

In [17]:
ratings = pd.read_csv('ratings.csv', encoding='latin-1', sep=',|:')
ratings.head()

/tmp/ipykernel_42225/3026495763.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('ratings.csv', encoding='latin-1', sep=',|:')


,UserId,ItemId,Rating
0,4baf0ac888,8b05db84f2,3
1,f3364fd7e6,8b05db84f2,2
2,72a8982204,8b05db84f2,5
3,d0171f7a9d,e40cec3cbe,5
4,e6dad08499,e40cec3cbe,5


In [15]:
mean = ratings.groupby(['UserId'], as_index = False, sort = False).mean()
mean.head()

,UserId,Rating
0,4baf0ac888,3.75
1,f3364fd7e6,1.50
2,72a8982204,5.00
3,d0171f7a9d,5.00
4,e6dad08499,4.00


In [19]:
ratings = pd.merge(ratings, mean, on='UserId', how='left', sort=False)
ratings['centered_rating'] = ratings['Rating_x']-ratings['Rating_y']
ratings.head()

,UserId,ItemId,Rating_x,Rating_y,centered_rating
0,4baf0ac888,8b05db84f2,3,3.75,-0.75
1,f3364fd7e6,8b05db84f2,2,1.50,0.50
2,72a8982204,8b05db84f2,5,5.00,0.00
3,d0171f7a9d,e40cec3cbe,5,5.00,0.00
4,e6dad08499,e40cec3cbe,5,4.00,1.00


In [65]:
item_to_user = {}
user_to_item = {}
for (_, row) in ratings.iterrows():
    user_id = row['UserId']
    item_id = row['ItemId']
    rating = row['centered_rating']
    if item_id not in item_to_user: item_to_user[item_id] = {}
    item_to_user[item_id][user_id] = rating
    if user_id not in user_to_item: user_to_item[user_id] = {}
    user_to_item[user_id][item_id] = rating

test_user = 'f3364fd7e6'
test_user_items = np.array(list(user_to_item[test_user].values())).astype(float)
denominator1 =  np.sqrt(sum(np.square(test_user_items)))
cosine_similarity = [(test_user, 1)]

for (user_id, items_map) in user_to_item.items():
    if user_id == test_user: continue
    numerator = [x*user_to_item[test_user][item_id] for (item_id, x) in items_map.items() if item_id in user_to_item[test_user]]
    denominator2 = np.sqrt(np.sum(np.square(list(items_map.values())).astype(float)))
    numerator_sum = np.sum(numerator)
    costheta = numerator_sum / (denominator1*denominator2) if numerator_sum != 0. else 0.
    cosine_similarity.append((user_id, costheta))


cosine_similarity.sort(key=lambda x: (x[1], x[0]), reverse=True)
cosine_similarity[:15]




[('test_user', 1),
 ('08750e4950', 0.577350269189626),
 ('2d21554615', 0.5773502691896255),
 ('2499fe7714', 0.5669467095138409),
 ('e26fbf09e6', 0.2886751345948126),
 ('28523fc4c6', 0.17739371879672491),
 ('193778e207', 0.08451542547285162),
 ('fffffc52fc', 0.0),
 ('ffffec2967', 0.0),
 ('ffffcdee5f', 0.0),
 ('ffffbeb8e0', 0.0),
 ('ffff829476', 0.0),
 ('ffff7821c4', 0.0),
 ('ffff722d66', 0.0),
 ('ffff5a7502', 0.0)]

In [64]:
cosine_similarity = []
for (user_id, items_map) in user_to_item.items():
    if user_id == test_user: continue
    numerator = [x*user_to_item[test_user][item_id] for (item_id, x) in items_map.items() if item_id in user_to_item[test_user]]
    # print(items_map)
    # print(np.nansum(np.sqrt(np.square(list(items_map.values())))))
    denominator2 = np.sqrt(np.sum(np.square(list(items_map.values()))))
    numerator_sum = np.sum(numerator)
    costheta = numerator_sum / (denominator1*denominator2) if numerator_sum != 0. else 0.
    # if(denominator2 == 0. and np.nansum(numerator) != 0.):
    # if(denominator2 == 0. ):
    #         print(items_map)
    #         # print("Denominator1", denominator1 )
    #         # print("Denominator2", denominator2 )
    #         # print("NumeratorSum", np.nansum(numerator) )
    #         break
    # break
    cosine_similarity.append((user_id, costheta))


cosine_similarity.sort(key=lambda x: (x[1], x[0]), reverse=True)
cosine_similarity[:15]

[('08750e4950', 0.577350269189626),
 ('2d21554615', 0.5773502691896255),
 ('2499fe7714', 0.5669467095138409),
 ('e26fbf09e6', 0.2886751345948126),
 ('28523fc4c6', 0.17739371879672491),
 ('193778e207', 0.08451542547285162),
 ('fffffc52fc', 0.0),
 ('ffffec2967', 0.0),
 ('ffffcdee5f', 0.0),
 ('ffffbeb8e0', 0.0),
 ('ffff829476', 0.0),
 ('ffff7821c4', 0.0),
 ('ffff722d66', 0.0),
 ('ffff5a7502', 0.0),
 ('ffff33af4a', 0.0)]

In [36]:
cosine_similarity.append(('my_test', 22.))
cosine_similarity.sort(key=lambda x: (x[1], x[0]), reverse=True)
cosine_similarity[:14]

[('my_test', 22.0),
 ('test_user', 1),
 ('72a8982204', nan),
 ('d0171f7a9d', nan),
 ('fa8e618830', 0.0),
 ('fa154404a9', 0.0),
 ('f50191053a', 0.0),
 ('f45e03186b', 0.0),
 ('f3d22ec598', 0.0),
 ('ea381ccbf6', 0.0),
 ('e6dad08499', 0.0),
 ('dd9d23b955', 0.0),
 ('d8e882dee0', 0.0),
 ('c506637567', 0.0)]